In [3]:
from dbt_utils import get_sql_improved

sql0 = get_sql_improved(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
sql1 = get_sql_improved(model='event_id_url', vars={'feature_table': 'basic_features'})
sql2 = get_sql_improved(model='event_id_url', vars={'feature_table': 'basic_features'}, depth_ephemeral_models=5)

print(sql1)

model.objectiv.event_id_url ephemeral
model.objectiv.event_id_context_type ephemeral

select
    event_id,
    context ->> 'id' as url
from "objectiv"."public"."event_id_context_type"
where context_type = 'WebDocumentContext'
-- todo: make sure event_id is unique


In [4]:
print(sql2)


with __dbt__CTE__event_id_context_type as (

with data as (
    select *
    from "objectiv"."public"."data"
), id_context as (
    select data.event_id as event_id,
           ctxs.rn as context_number,
           ctxs.ctx as context
    from data
    left join lateral json_array_elements(data.value->'contexts') with ordinality as ctxs (ctx, rn) on true
), id_context_type_context as (
    select event_id                                              as event_id,
           context_number,
           json_array_elements_text(context -> '_context_types') as context_type,
           context                                               as context
    from id_context
)
select
    event_id,
    context_number,
    context_type,
    context
from id_context_type_context
order by event_id, context_number
)select
    event_id,
    context ->> 'id' as url
from __dbt__CTE__event_id_context_type
where context_type = 'WebDocumentContext'
-- todo: make sure event_id is unique


In [2]:
import psycopg2
import psycopg2.extras
import os
POSTGRES_USER =     os.environ.get('POSTGRES_USER',     'objectiv')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD', '0bj3ctiv')
POSTGRES_DB =       os.environ.get('POSTGRES_DB',       'objectiv')
POSTGRES_HOSTNAME = os.environ.get('POSTGRES_HOSTNAME', 'localhost')
POSTGRES_PORT =     os.environ.get('POSTGRES_PORT',     '5432')

connection = psycopg2.connect(user=POSTGRES_USER,
                            password=POSTGRES_PASSWORD,
                            host=POSTGRES_HOSTNAME,
                            port=POSTGRES_PORT,
                            database=POSTGRES_DB)

# This file should be mapped to a volume in the docker container,
# so changes to this file from within the container can be persisted.

with connection:
    with connection.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
        cursor.execute(sql0)
        rows = cursor.fetchall()


In [3]:
rows

[['Pretty(NavigationContext,navigation),(OverlayContext,navigation-drawer),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  1,
  1],
 ['Pretty(NavigationContext,navigation),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  1,
  1],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ButtonContext,prev),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  5],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,5o7WEv5Q5ZE),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,BeyEGebJ1l4),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,BodkSiBOw3A),(WebDocumentContext,rod-web)',
  datetime.date(2021, 5, 6),
  1,
  2,
  3],
 ['Pretty(SectionContext,home),(SectionContext,for-you),(ItemContext,cc91EfoBh8A),(WebDocumentContext,rod-

In [4]:
sql


'with __dbt__CTE__extracted_contexts as (\nSELECT *,\n        value->>\'event\' AS event,\n        JSON_EXTRACT_PATH(value, \'global_contexts\') AS global_contexts,\n        JSON_EXTRACT_PATH(value, \'location_stack\') AS location_stack,\n        JSON_EXTRACT_PATH(value, \'time\') AS time,\n        JSON_EXTRACT_PATH(value, \'events\') AS events\n FROM "objectiv"."public"."data"\n),  __dbt__CTE__hashed_features as (\nWITH selected_stacks AS\n(\nSELECT   event_id,\n         Array_to_string(Array_agg(Cast(x AS TEXT)),\',\') AS stack_selection,\n         Array_to_json(Array_agg(Row_to_json(x))) as selected_stack_location\nFROM     __dbt__CTE__extracted_contexts,\n         json_to_recordset(location_stack) AS x(_context_type text,id text)\nGROUP BY event_id\nORDER BY event_id\n)\nSELECT *,\n       md5(concat(stack_selection,event)) as feature_hash\nFROM __dbt__CTE__extracted_contexts\nJOIN selected_stacks USING (event_id)\n),  __dbt__CTE__sessionized_data as (\nwith session_starts as (\n   